# Baseline Eval


## Summary
- Purpose: establish the prompting-only baseline for NL->SQL (`k=0` and `k=3`).
- Scope: runs on the fixed 200-item ClassicModels benchmark for paired comparisons.
- Outputs: baseline run JSON files under `results/baseline/` for downstream analysis.


In [ ]:
%%bash
set -e

# shared colab setup script keeps package versions aligned across notebooks.
if [ ! -f scripts/colab_setup.sh ] && [ -d /content ]; then
  if [ ! -d /content/NLtoSQL ]; then
    git clone https://github.com/MacKenzieOBrian/NLtoSQL.git /content/NLtoSQL
  fi
  cd /content/NLtoSQL
fi

bash scripts/colab_setup.sh


## 0) Bootstrap


In [ ]:
import os, sys, shutil
from pathlib import Path

# If this notebook is opened directly in Colab (not from a cloned repo), clone the repo first.
if Path("data/classicmodels_test_200.json").exists() is False and Path("/content").exists():
    repo_dir = Path("/content/NLtoSQL")
    if repo_dir.exists():
        shutil.rmtree(repo_dir)
    !git clone https://github.com/MacKenzieOBrian/NLtoSQL.git "{repo_dir}"
    os.chdir(repo_dir)

# Ensure repo root is on sys.path for `import nl2sql`
sys.path.insert(0, os.getcwd())
print("cwd:", os.getcwd())


## 1) Install


In [ ]:
try:
    import google.colab  # noqa: F401
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install -r requirements.txt
else:
    print("Not in Colab; ensure requirements are installed.")


In [ ]:
# Colab-only: authenticate with GCP (safe to skip locally)
try:
    from google.colab import auth
except ModuleNotFoundError:
    auth = None

if auth:
    auth.authenticate_user()
else:
    print("Not running in Colab; ensure ADC or service account auth is configured.")


In [ ]:
# Hugging Face auth (gated model)
import os

hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if hf_token:
    os.environ["HUGGINGFACE_HUB_TOKEN"] = hf_token
    print("Using HF token from env")
else:
    try:
        from huggingface_hub import notebook_login
        notebook_login()
    except Exception as e:
        print("HF auth not configured:", e)


## 2) Data + DB


In [ ]:
import json
from getpass import getpass

INSTANCE_CONNECTION_NAME = os.getenv("INSTANCE_CONNECTION_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_NAME = os.getenv("DB_NAME", "classicmodels")

if not INSTANCE_CONNECTION_NAME:
    INSTANCE_CONNECTION_NAME = input("Enter INSTANCE_CONNECTION_NAME: ").strip()
if not DB_USER:
    DB_USER = input("Enter DB_USER: ").strip()
if not DB_PASS:
    DB_PASS = getpass("Enter DB_PASS: ")

print("Using DB:", DB_NAME)

test_set = json.loads(open("data/classicmodels_test_200.json", "r", encoding="utf-8").read())
print("Loaded test items:", len(test_set))


In [ ]:
from nl2sql.db import create_engine_with_connector

engine, connector = create_engine_with_connector(
    instance_connection_name=INSTANCE_CONNECTION_NAME,
    user=DB_USER,
    password=DB_PASS,
    db_name=DB_NAME,
)

print("Engine ready")


## 3) Load Model


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
# this is the base model for baseline prompting runs.
# keep this fixed when comparing prompt and k-shot settings.
# provenance: model is fixed per sweep so deltas reflect prompt/shot changes, not model swaps.

print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(MODEL_ID, token=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

# Try 4-bit loading (fallback to fp/bf16)
# provenance: 4-bit nf4 path follows qlora-style inference to stay within colab memory.
try:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )
    print("Attempting 4-bit quantized load...")
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        token=True,
    )
except Exception as e:
    print("4-bit load failed, falling back. Error:")
    print(e)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        token=True,
    )

# transformers generation docs: temperature/top_p are sampling controls.
# deterministic eval policy: do_sample=false with neutral temperature/top_p.
model.generation_config.do_sample = False
model.generation_config.temperature = 1.0
model.generation_config.top_p = 1.0
model.generation_config.top_k = 50
model.generation_config.num_beams = 1

print("Model device:", model.device)


## 4) Schema Context


In [ ]:
from nl2sql.schema import build_schema_summary

SCHEMA_SUMMARY = build_schema_summary(engine, db_name=DB_NAME, max_cols_per_table=50)
print("Schema summary built (chars):", len(SCHEMA_SUMMARY))


## 5) Run Controls


In [ ]:
# Orchestration helpers for baseline sweep runs.
# Core NL->SQL logic stays in nl2sql imports below.
import re
import subprocess
import shutil
from functools import lru_cache
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd

from nl2sql.eval import eval_run
from nl2sql.db import create_engine_with_connector
import nl2sql.prompting as prompting_mod

DEFAULT_SYSTEM_INSTRUCTIONS = prompting_mod.SYSTEM_INSTRUCTIONS


from nl2sql.experiment_helpers import (
    exemplar_pool_for_strategy,
    make_prompt_variants,
    model_alias_from_id,
    schema_variant_text,
)

# shared helpers keep baseline/qlora notebook behavior aligned.
_model_alias_from_id = model_alias_from_id
PROMPT_VARIANTS = make_prompt_variants(DEFAULT_SYSTEM_INSTRUCTIONS)

# Capture git commit for run provenance (optional).
try:
    commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"]).decode().strip()
except Exception:
    commit = "unknown"

# Base metadata attached to each saved run JSON.
run_metadata_base = {
    "commit": commit,
    "model_id": MODEL_ID,
    "model_alias": _model_alias_from_id(MODEL_ID),
    "notebook": "02_baseline_prompting_eval.ipynb",
    "method": "baseline",
}

# Ensure output root exists before writing run artifacts.
Path("results/baseline").mkdir(parents=True, exist_ok=True)



# Main sweep runner: executes k/seed grid and writes JSON+CSV artifacts.
def run_baseline_grid(
    *,
    k_values: list[int],
    seeds: list[int],
    run_tag: str,
    prompt_variant: str,
    schema_variant: str,
    exemplar_strategy: str,
    limit: int | None = None,
    copy_canonical: bool = True,
    model_alias: str | None = None,
    enable_ts_for_k: set[int] | None = None,
    ts_n: int = 10,
    ts_prefix: str = "classicmodels_ts",
    ts_max_rows: int = 500,
):
    # Basic reproducibility checks.
    if not seeds:
        raise ValueError("Provide at least one seed")

    if prompt_variant not in PROMPT_VARIANTS:
        raise ValueError(f"Unknown PROMPT_VARIANT: {prompt_variant}")

    # Run directory is timestamped for traceability and collision safety.
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%SZ")
    run_dir = Path("results/baseline/runs") / f"{run_tag}_{ts}"
    run_dir.mkdir(parents=True, exist_ok=True)

    schema_used = schema_variant_text(SCHEMA_SUMMARY, schema_variant)
    exemplar_pool = exemplar_pool_for_strategy(test_set, exemplar_strategy)
    resolved_model_alias = model_alias or run_metadata_base.get("model_alias") or _model_alias_from_id(MODEL_ID)

    ts_enabled_k = set(enable_ts_for_k or set())
    ts_suite_db_names = (
        [f"{ts_prefix}_{i:02d}" for i in range(1, ts_n + 1)]
        if ts_enabled_k and ts_n > 0
        else None
    )

    # Lazy connector cache for optional TS evaluation databases.
    ts_connectors = {}

    @lru_cache(maxsize=32)
    def _make_engine_cached(db_name: str):
        eng, conn = create_engine_with_connector(
            instance_connection_name=INSTANCE_CONNECTION_NAME,
            user=DB_USER,
            password=DB_PASS,
            db_name=db_name,
        )
        ts_connectors[db_name] = conn
        return eng

    def _make_engine_fn(db_name: str):
        return _make_engine_cached(db_name)

    rows = []
    primary_seed = seeds[0]  # first seed is used for canonical copies

    # Prompt override is temporary and restored in finally.
    old_prompt = prompting_mod.SYSTEM_INSTRUCTIONS
    prompting_mod.SYSTEM_INSTRUCTIONS = PROMPT_VARIANTS[prompt_variant]

    try:
        for k in k_values:
            # Keep k=0 tied to primary seed for canonical zero-shot consistency.
            seed_list = [primary_seed] if k == 0 else seeds
            for seed in seed_list:
                save_path = run_dir / f"results_k{k}_seed{seed}.json"

                run_meta = dict(run_metadata_base)
                run_meta.update({
                    "run_tag": run_tag,
                    "k": k,
                    "seed": seed,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                    "model_alias": resolved_model_alias,
                    "ts_enabled": bool(k in ts_enabled_k),
                    "ts_for_k_values": sorted(ts_enabled_k),
                    "ts_n": ts_n if ts_suite_db_names else 0,
                    "eval_profile": "model_only_raw",
                })

                items = eval_run(
                    test_set=test_set,
                    exemplar_pool=exemplar_pool,
                    k=k,
                    limit=limit,
                    seed=seed,
                    engine=engine,
                    model=model,
                    tokenizer=tok,
                    schema_summary=schema_used,
                    save_path=str(save_path),
                    run_metadata=run_meta,
                    ts_suite_db_names=ts_suite_db_names if k in ts_enabled_k else None,
                    ts_make_engine_fn=_make_engine_fn if k in ts_enabled_k else None,
                    ts_max_rows=ts_max_rows,
                    avoid_exemplar_leakage=True,
                )

                # Aggregate per-run rates for quick table summaries.
                n = len(items)
                va = sum(int(x.va) for x in items) / max(n, 1)
                em = sum(int(x.em) for x in items) / max(n, 1)
                ex = sum(int(x.ex) for x in items) / max(n, 1)
                ts_values = [int(x.ts) for x in items if getattr(x, "ts", None) is not None]
                ts_rate = (sum(ts_values) / len(ts_values)) if ts_values else None

                rows.append({
                    "run_tag": run_tag,
                    "prompt_variant": prompt_variant,
                    "schema_variant": schema_variant,
                    "exemplar_strategy": exemplar_strategy,
                    "exemplar_pool_size": len(exemplar_pool),
                    "k": k,
                    "seed": seed,
                    "n": n,
                    "va_rate": va,
                    "em_rate": em,
                    "ex_rate": ex,
                    "ts_rate": ts_rate,
                    "ts_n": len(ts_values),
                    "json_path": str(save_path),
                })

                if seed == primary_seed and k in {0, 3}:
                    if copy_canonical:
                        target = (
                            Path("results/baseline/results_zero_shot_200.json")
                            if k == 0
                            else Path("results/baseline/results_few_shot_k3_200.json")
                        )
                        target.parent.mkdir(parents=True, exist_ok=True)
                        shutil.copy2(save_path, target)
                        print(f"Updated canonical file: {target}")

    finally:
        for conn in ts_connectors.values():
            try:
                conn.close()
            except Exception:
                pass
        prompting_mod.SYSTEM_INSTRUCTIONS = old_prompt

    # Save per-run table and per-k mean/std summary.
    df = pd.DataFrame(rows).sort_values(["k", "seed"]).reset_index(drop=True)
    df.to_csv(run_dir / "grid_summary.csv", index=False)

    agg = (
        df.groupby(["prompt_variant", "schema_variant", "exemplar_strategy", "k"], as_index=False)
        .agg(
            runs=("seed", "count"),
            va_mean=("va_rate", "mean"),
            va_std=("va_rate", "std"),
            em_mean=("em_rate", "mean"),
            em_std=("em_rate", "std"),
            ex_mean=("ex_rate", "mean"),
            ex_std=("ex_rate", "std"),
            ts_mean=("ts_rate", "mean"),
            ts_std=("ts_rate", "std"),
        )
    )
    agg.to_csv(run_dir / "grid_summary_by_k.csv", index=False)

    print("Saved grid run to:", run_dir)
    return df, agg, run_dir


### 5A) Run Plan


In [ ]:
# ============================
# Plan-driven baseline controls
# ============================
# Edit only this block for most experiments.
# Keep one RUN_PLAN active per execution to isolate effects clearly.
RUN_PLAN = "quick"  # quick | k5_core | seed_backfill | full_sweep | ts_k3 | custom
# run_plan picks which predefined grid to execute.
# use custom when you need one exact run.

PROMPT_VARIANT = "default"
SCHEMA_VARIANT = "full"
EXEMPLAR_STRATEGY = "all"
# these three toggles control ablation axes. keep them fixed for fair model comparison.
# provenance: these axes map directly to methodology sections (prompt, schema context, exemplar pool).

MODEL_ALIAS = _model_alias_from_id(MODEL_ID)
COPY_CANONICAL = False

# optional reliability extension path (keep false for primary hypothesis runs)
USE_OPTIONAL_RELIABILITY = False
GENERATION_CONSTRAINED = USE_OPTIONAL_RELIABILITY
GENERATION_EXTRACT_SELECT = USE_OPTIONAL_RELIABILITY
GENERATION_STOP_ON_SEMICOLON = USE_OPTIONAL_RELIABILITY
APPLY_SQL_GUARDRAILS = USE_OPTIONAL_RELIABILITY
APPLY_POSTPROCESS = USE_OPTIONAL_RELIABILITY

TS_N = 10
# provenance: ts_n=10 mirrors execution-consistency stress tests used in final reporting.
TS_PREFIX = "classicmodels_ts"
TS_MAX_ROWS = 500

CUSTOM_PLAN = {
    "k_values": [0, 3],
    "seeds": [7],
    "run_tag": "baseline_custom",
    "enable_ts": False,
}

# Resolve plan -> concrete k/seed/run_tag values.
if RUN_PLAN == "quick":
    K_VALUES = [0, 3]
    SEEDS = [7]
    RUN_TAG = "baseline_main"
    ENABLE_TS = False
elif RUN_PLAN == "k5_core":
    K_VALUES = [5]
    SEEDS = [7]
    RUN_TAG = "baseline_k5_core"
    ENABLE_TS = False
elif RUN_PLAN == "seed_backfill":
    K_VALUES = [0, 3, 5]
    SEEDS = [17, 27]
    RUN_TAG = "baseline_seed_backfill"
    ENABLE_TS = False
elif RUN_PLAN == "full_sweep":
    K_VALUES = [0, 3, 5]
    SEEDS = [7, 17, 27]
    RUN_TAG = "baseline_e1_k_sweep"
    ENABLE_TS = False
elif RUN_PLAN == "ts_k3":
    K_VALUES = [3]
    SEEDS = [7]
    RUN_TAG = "baseline_ts_k3"
    ENABLE_TS = True
elif RUN_PLAN == "custom":
    K_VALUES = list(CUSTOM_PLAN["k_values"])
    SEEDS = list(CUSTOM_PLAN["seeds"])
    RUN_TAG = str(CUSTOM_PLAN["run_tag"])
    ENABLE_TS = bool(CUSTOM_PLAN["enable_ts"])
else:
    raise ValueError(f"Unknown RUN_PLAN: {RUN_PLAN}")

# TS is typically enabled only for k=3 semantic robustness checks.
TS_FOR_K_VALUES = [3]

# Run card is a quick sanity check before launching a long run.
print("Baseline run card:")
print({
    "run_plan": RUN_PLAN,
    "run_tag": RUN_TAG,
    "model_id": MODEL_ID,
    "model_alias": MODEL_ALIAS,
    "k_values": K_VALUES,
    "seeds": SEEDS,
    "prompt_variant": PROMPT_VARIANT,
    "schema_variant": SCHEMA_VARIANT,
    "exemplar_strategy": EXEMPLAR_STRATEGY,
    "enable_ts": ENABLE_TS,
    "optional_reliability": USE_OPTIONAL_RELIABILITY,
})


In [ ]:
# Execute selected sweep.
baseline_grid, baseline_by_k, baseline_run_dir = run_baseline_grid(
    k_values=K_VALUES,
    seeds=SEEDS,
    run_tag=RUN_TAG,
    prompt_variant=PROMPT_VARIANT,
    schema_variant=SCHEMA_VARIANT,
    exemplar_strategy=EXEMPLAR_STRATEGY,
    limit=None,
    copy_canonical=COPY_CANONICAL,
    model_alias=MODEL_ALIAS,
    enable_ts_for_k=set(TS_FOR_K_VALUES) if ENABLE_TS else None,
    ts_n=TS_N,
    ts_prefix=TS_PREFIX,
    ts_max_rows=TS_MAX_ROWS,
    generation_constrained=GENERATION_CONSTRAINED,
    generation_extract_select=GENERATION_EXTRACT_SELECT,
    generation_stop_on_semicolon=GENERATION_STOP_ON_SEMICOLON,
    apply_sql_guardrails=APPLY_SQL_GUARDRAILS,
    apply_postprocess=APPLY_POSTPROCESS,
)

print("\nPer-run rows:")
display(baseline_grid)
print("\nPer-k summary (mean/std across seeds):")
display(baseline_by_k)


In [ ]:
# 6) Quick canonical summary (reads saved k=0 and k=3 JSON outputs)
import json

zero = json.loads(open("results/baseline/results_zero_shot_200.json", "r", encoding="utf-8").read())
few  = json.loads(open("results/baseline/results_few_shot_k3_200.json", "r", encoding="utf-8").read())

print("Zero-shot:", "VA", round(zero["va_rate"], 3), "EM", round(zero.get("em_rate", 0.0), 3), "EX", round(zero["ex_rate"], 3))
print("Few-shot:",  "VA", round(few["va_rate"], 3),  "EM", round(few.get("em_rate", 0.0), 3),  "EX", round(few["ex_rate"], 3))
